In [1]:
import numpy as np
from draughts1 import *

from time import time

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from keras import backend as K

Scan.set("variant", "normal")
Scan.set("book", "false")
Scan.set("book-ply", "4")
Scan.set("book-margin", "4")
Scan.set("ponder", "false")
Scan.set("threads", "1")
Scan.set("tt-size", "24")
Scan.init()

In [ ]:
import pandas as pd

nmoves = pd.read_csv('normal_moves.csv')

In [ ]:
nmoves[(nmoves['from']==3)&(nmoves['to']==8)]['move_id'].iloc[0]

In [ ]:
nmoves

In [ ]:
games0 = parse_pdn_file('../games/wiersma.pdn')
games1 = parse_pdn_file('2017.pdn')

In [17]:
aa = np.array([1,2,1,1,4,5,6,1])
result = np.where(aa == 1)
result

(array([0, 2, 3, 7], dtype=int64),)

In [ ]:
def convertPosToID(pos):

    text = print_position(pos, False, True)

    chr_player = 'o' if pos.is_white_to_move() else 'x'
    chr_other = 'x' if pos.is_white_to_move() else 'o'

    currentplayer_man = []
    other_man = []
    currentplayer_king = []
    other_king = []

    for a,i in enumerate(text[:-1]):
        if a%10<5:
            currentplayer_man.append(0)
            other_man.append(0)
            currentplayer_king.append(0)
            other_king.append(0)
        
        currentplayer_man.append(1 if i==chr_player else 0)
        other_man.append(1 if i==chr_other else 0)
        currentplayer_king.append(1 if i==chr_player.upper() else 0)
        other_king.append(1 if i==chr_other.upper() else 0)
        
        if a%10>=5:
            currentplayer_man.append(0)
            other_man.append(0)
            currentplayer_king.append(0)
            other_king.append(0)
            

    position = np.concatenate((currentplayer_man, other_man, currentplayer_king, other_king), axis=0)
    return np.array(position)

In [ ]:
def get_pos(moves, hh):
    posIDs = []
    evals = []
    hashs = []
    pos = start_position()
    for move in moves:
        is_white = pos.white_man_count()==3 and pos.black_man_count()==2 and pos.white_king_count()==0 and pos.black_king_count()==0 and pos.is_white_to_move()
        is_black = pos.white_man_count()==2 and pos.black_man_count()==3 and pos.white_king_count()==0 and pos.black_king_count()==0 and not pos.is_white_to_move()
        if is_white or is_black:
            hk = hash_key(pos)
            if hk not in hh:
                posID = convertPosToID(pos)
                posIDs.append(posID)
                
                evals.append(eval_position(pos))
                hashs.append(hk)
        pos = pos.succ(move)
    return posIDs, evals, hashs

In [ ]:
def get_pos2(moves, hh):
    posIDs = []
    evals = []
    hashs = []
    pos = start_position()
    for move in moves:
        if pos.white_man_count()==3 and pos.black_man_count()==3 and pos.white_king_count()==0 and pos.black_king_count()==0:
            hk = hash_key(pos)
            if hk not in hh:
                posID = pos_to_numpy1(pos)
                posIDs.append(posID)
                
                evals.append(eval_position(pos))
                hashs.append(hk)
        pos = pos.succ(move)
    return posIDs, evals, hashs

In [ ]:
for game in games1:
    print(game.event)

In [ ]:
games1

In [ ]:
posIDs = []
evals = []
hashs = []
for game in games1:
    p,e,h  = get_pos2(game.moves,hashs)
    posIDs += p
    evals += e
    hashs += h

In [ ]:
len(hashs)

In [ ]:
np.save("posIDs",posIDs)
np.save("evals",evals)
np.save("hashs",hashs)

In [ ]:
posIDs = np.load("posIDs.npy")

In [ ]:
posIDs = np.array(posIDs)
evals = np.array(evals)

In [ ]:
posIDs.shape

In [ ]:
posIDs = np.load("posIDs.npy")
evals = np.load("evals.npy")
hashs = np.load("hashs.npy")

In [ ]:
start_time = time()
Scan.set("bb-size", "5")
Scan.update()

X5 = []
y5 = []
n_out = [0,0,0,0]
enumerator = EGDBEnumerator(3, 2, 0, 0)

prev_pos = start_position()
flag = False

while enumerator.next():
    pos = enumerator.position()
    if flag and hash_key(pos) not in hashs:
        flag = False
        X5.append(convertPosToID(pos))
        y5.append(eval_position(pos))
    if hash_key(pos) in hashs:
        X5.append(convertPosToID(prev_pos))
        y5.append(eval_position(prev_pos))
        flag = True
    prev_pos = pos

X5 = np.array(X5)
y5 = np.array(y5)

print(time() - start_time)
print(n_out)

In [ ]:
X5.shape

In [ ]:
posIDs[0]

In [ ]:
X5[0]

In [ ]:
start_time = time()
Scan.set("bb-size", "5")
Scan.update()

enumerator = EGDBEnumerator(3, 2, 0, 0)

cID = []
count = 0
while enumerator.next():
    pos = enumerator.position()
    if hash_key(pos) in hashs:
        cID.append(count)
    count+=1
        

print(time() - start_time)

In [ ]:
len(cID)

In [ ]:
np.save("cID",cID)

In [ ]:

cID = np.load("cID.npy")

In [ ]:
cID = np.sort(cID)
cID

In [ ]:
def scan_search1(pos, max_depth, max_time):
    Scan.set("bb-size", "4")
    Scan.update()
    si = SearchInput()
    si.move = True
    si.book = False
    si.depth = max_depth
    si.nodes = 1000000000000
    si.time = max_time
    si.input = True
    
    so = SearchOutput()
    node = make_node(pos)
    
    search(so, node, si)
        
    return so.score, so.move

In [ ]:
errID[-5:]

In [ ]:
count

In [ ]:
Scan.set("bb-size", "5")
Scan.update()

enumerator = EGDBEnumerator(3, 2, 0, 0)

count = 0
closestID = 0

errPos = []
errProbe = []

while enumerator.next():
    if count%1000==0:
        print(int(count/1000),end=' ')
    if errID[closestID]==count:
        print(closestID, end=' ')
        pos = enumerator.position()
        errPos.append(pos)
        errProbe.append(EGDB.probe(pos))
        closestID += 1
    elif count>errID[closestID]+bound:
        closestID += 1
    count += 1
    if closestID>=len(errID):
        print('sada ', closestID)
        break

In [ ]:
len(errID)

In [ ]:
len(errProbe)

In [ ]:
import random

print(random.randint(0,1))

In [ ]:
bound = 200

start_time = time()
# X5 = []
y5_score = []
# y5_true = []
enumerator = EGDBEnumerator(3, 2, 0, 0)
# n_out = [0,0,0,0]

count = 0
closestID = 0
# convert_time = 0
# eval_time = 0
# pos_time = 0
# other_time = 0
# if_time = 0
cc = 0

while enumerator.next():
    if count%1000==0:
        print(int(count/1000),end=' ')
#     other_time += time() - start_time
#     start_time = time()
    if cID[closestID]-bound<=count and cID[closestID]+bound>=count:
        cc += 1
#         if_time += time() - start_time
#         start_time = time()
        pos = enumerator.position()
#         pos_time += time() - start_time
#         start_time = time()
#         X5.append(pos_to_numpy1(pos))
#         convert_time += time() - start_time
#         start_time = time()
        score, _ = scan_search1(play_forced_moves(pos),5,5)
        y5_score.append(score)
#         Scan.set("bb-size", "5")
#         Scan.update()
#         probe_out = EGDB.probe(pos)
#         output = np.zeros(4)
#         output[probe_out] = 1
#         y5_true.append(output)
#         eval_time += time() - start_time
#         start_time = time()
    elif count>cID[closestID]+bound:
        closestID += 1
#         if_time += time() - start_time
#         start_time = time()
#     else:
#         if_time += time() - start_time
#         start_time = time()
        
    count+=1
    if closestID>=len(cID):
        break

# X5 = np.array(X5)
# y5_score = np.array(y5_score)
# y5_true = np.array(y5_true)

print(time() - start_time)
# print(n_out)

In [ ]:
len(X5)

In [ ]:
len(y5_score)

In [ ]:
y5g_true

In [ ]:
y5_score[:10]

In [ ]:
y5_score1[:10]

In [ ]:
y5_score2[:10]

In [ ]:
len(y5_score2)

In [ ]:
len(y5_true)

In [ ]:
len(X5)

In [ ]:
np.save("y5_score_xxs",y5_score)
np.save("y5_true_xxs",y5_true)
np.save("X5_xxs",X5)

In [ ]:
y5_score_prev = np.load("y5_score.npy")

In [ ]:
len(y5_score_prev)

In [ ]:
y5_score

In [ ]:
y5_score_prev

In [ ]:
Scan.set("bb-size", "5")
Scan.update()

bound = 200

start_time = time()
# X5 = []
# y5_score = []
# y5_true = []
enumerator = EGDBEnumerator(3, 2, 0, 0)
# n_out = [0,0,0,0]

count = 0
closestID = 0
# convert_time = 0
# eval_time = 0
# pos_time = 0
# other_time = 0
# if_time = 0
# captureID = []
# noncapID = []

# threatID = []
# threatP = []
# nonthreatID = []

errID = []
# errID_probe = []
# nonerrID = []
cy = 0

while enumerator.next():
    if count%1000==0:
        print(int(count/1000),end=' ')
#     other_time += time() - start_time
#     start_time = time()
    if cID[closestID]-bound<=count and cID[closestID]+bound>=count:
#         if_time += time() - start_time
#         start_time = time()
        pos = enumerator.position()
#         score, _ = scan_search(pos,15,5)
        if y5_score[cy]==-10001:
            errID.append(count)
#             errID_probe.append(EGDB.probe(pos))
#         else:
#             nonerrID.append(count)
        cy += 1
#         if pos.is_capture():
#             captureID.append(count)
#         else:
#             noncapID.append(count)
#         if pos.is_threat():
#             threatID.append(count)
#             threatP.append(EGDB.probe(pos))
#         else:
#             nonthreatID.append(count)
#         pos_time += time() - start_time
#         start_time = time()
#         X5.append(convertPosToID(pos))
#         convert_time += time() - start_time
#         start_time = time()
#         score, _ = scan_search(pos,15,5)
#         y5_score.append(score)
#         probe_out = EGDB.probe(pos)
#         output = np.zeros(4)
#         output[probe_out] = 1
#         n_out[probe_out] += 1
#         y5_true.append(output)
#         eval_time += time() - start_time
#         start_time = time()
    elif count>cID[closestID]+bound:
        closestID += 1
#         if_time += time() - start_time
#         start_time = time()
#     else:
#         if_time += time() - start_time
#         start_time = time()
        
    count+=1
    if closestID>=len(cID):
        break

# X5 = np.array(X5)
# y5_score = np.array(y5_score)
# y5_true = np.array(y5_true)

print(time() - start_time)
# print(n_out)

In [ ]:
len(errID_probe)

In [ ]:
np.unique(np.array(errID_probe),return_counts=True)

In [ ]:
len(captureID)

In [ ]:
len(noncapID)

In [ ]:
len(threatID)

In [ ]:
np.unique(np.array(threatP),return_counts=True)

In [ ]:
[pos_time,convert_time,eval_time,if_time,other_time]

In [ ]:
len(X5)

In [ ]:
len(y5_true)

In [ ]:
np.unique(y5)

In [ ]:
# X5g = X5.reshape(len(X5),4,10,10)
y5g = []
n_out = [0,0,0,0]
for i in y5:
    if i<-139:
        n_out[1] += 1
        y5g.append([0,1,0,0])
    elif i>139:
        n_out[2] += 1
        y5g.append([0,0,1,0])
    else:
        n_out[0] += 1
        y5g.append([1,0,0,0])
n_out

In [ ]:
y5g = np.array(y5g)

In [ ]:
X5g.shape

In [ ]:
# np.save("X5g2",X5g)
np.save("y5g2",y5g)

In [ ]:
np.save("y5g_true",y5_true)

In [ ]:
np.save("y5_score",y5_score)

In [ ]:
y5g.shape

In [ ]:

X5g = np.load("X5g.npy")
# y5g = np.load("y5g.npy")
# y5g2 = np.load("y5g2.npy")
# y5_score = np.load("y5_score.npy")

In [ ]:
errID2 = np.load("errID.npy")

In [ ]:
errID

In [ ]:
errID2[:20]

In [ ]:
X5g.shape

In [ ]:
y5_true

In [ ]:
y5g_true = np.load("y5g_true.npy")

In [ ]:
len(X5)

In [ ]:
y5g_true.shape

In [ ]:
X5n = []
y5n_true = []
closestID = 0
a = 0

enumerator = EGDBEnumerator(3, 2, 0, 0)
while enumerator.next():
    if a%1000==0:
        print(int(a/1000),end=' ')
    if a == nonerrID[closestID]:
        pos = enumerator.position()
        X5n.append(convertPosToID(pos))
        probe_out = EGDB.probe(pos)
        output = np.zeros(4)
        output[probe_out] = 1
        y5n_true.append(output)
        a += 1
        closestID += 1
    elif a>nonerrID[closestID]:
        closestID += 1
    else:
        a += 1
    if closestID>=len(nonerrID):
        print(closestID)
        break

In [ ]:
len(errID_probe)

In [ ]:
len(nonerrID)

In [ ]:
len(X5n)

In [ ]:
len(y5n)

In [ ]:
len(y5n_true)

In [ ]:
X5n = np.array(X5n)
y5n = np.array(y5n)
y5n_true = np.array(y5n_true)

In [ ]:
X5n = np.load("X5n.npy")
y5n = np.load("y5n.npy")
y5n_true = np.load("y5n_true.npy")

In [ ]:
X5g.shape

In [ ]:
len(y5n)

In [ ]:
len(y5_true)

In [ ]:
np.save("y5g",y5n)
np.save("y5g_true",y5_true)

In [ ]:
len(y5g_true)

In [ ]:
n_same = 0
n_res = np.zeros([4,4])
difPos = []

for i in range(len(y5n)):
    res = np.sum(y5n[i]*y5g_true[i])
    if res!=1:
        difPos.append(i)
    n_same += res
    n_res[list(y5n[i]).index(1)][list(y5g_true[i]).index(1)] += 1

In [ ]:
n_res

In [ ]:
len(difPos)/len(y5n)

In [ ]:
1-len(difPos)/len(y5n)

In [ ]:
n_same

In [ ]:
n_same/len(y5n)

In [ ]:
y5n = []
# errID = []
count = [0,0,0,0]
err_count = 0
for a,i in enumerate(y5_score):
    if i>=-100 and i<=100:
        count[0] += 1
        y5n.append([1,0,0,0])
    elif i>100:
        count[2] += 1
        y5n.append([0,0,1,0])
    else:
        count[1] += 1
        y5n.append([0,1,0,0])
    if i==-10001:
        err_count +=1
        
count

In [ ]:
np.save("y5n_xxs",y5n)

In [ ]:
np.save("X5n",X5n)
np.save("y5n",y5n)
np.save("nonerrID",nonerrID)
np.save("y5n_true",y5n_true)
np.save("errID",errID)
np.save("errID_probe",errID_probe)

In [ ]:
closestID = 0
a = 0
errPos = []

enumerator = EGDBEnumerator(3, 2, 0, 0)
while enumerator.next():
    if a%1000==0:
        print(int(a/1000),end=' ')
    if a == errID[closestID]:
        pos = enumerator.position()
        errPos.append(pos)
        a += 1
        closestID += 1
    elif a>errID[closestID]:
        closestID += 1
    else:
        a += 1
    if closestID>=len(errID):
        print(closestID)
        break

In [ ]:
display_position(errPos[0])

In [ ]:
closestID = 0
a = 0

enumerator = EGDBEnumerator(3, 2, 0, 0)
while enumerator.next():
    if a%1000==0:
        print(int(a/1000),end=' ')
    if a == nonerrID[closestID]:
        a += 1
        closestID += 1
    elif a>nonerrID[closestID]:
        closestID += 1
    else:
        a += 1
    if closestID>=len(nonerrID):
        print(closestID)
        break

In [ ]:
def create_model_cnn1():
    model = Sequential()
    model.add(Input(shape=(4, 10, 10)))
    model.add(Conv2D(64, 5, padding="same", activation="relu"))
    model.add(Conv2D(64, 5, padding="same", activation="relu"))
    model.add(Conv2D(128, 4, padding="same", activation="relu"))
    model.add(Conv2D(128, 4, padding="same", activation="relu"))
    model.add(MaxPooling2D(2))
    model.add(Conv2D(128, 3, padding="same", activation="relu"))
    model.add(Conv2D(128, 3, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
def create_model_cnn2():
    model = Sequential()
    model.add(Input(shape=(4, 10, 10)))
    model.add(Conv2D(256, 4, padding="same", activation="relu"))
    model.add(Conv2D(256, 4, padding="same", activation="relu"))
    model.add(MaxPooling2D(2))
    model.add(Conv2D(256, 3, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(4, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
model4_X5_f2_xs = create_model_cnn2()

In [ ]:
X5 = np.array(X5).reshape((len(X5), 4, 10, 10))


In [ ]:
X5.shape

In [ ]:
y5n = np.array(y5n)

In [ ]:
y5n.shape

In [ ]:
np.save("y5n_xs",y5n)

In [ ]:
X5xs = np.load("X5_xs.npy")
y5xs = np.load("y5n_xs.npy")

In [ ]:
X5 = np.load("X5_xxs.npy")
y5n = np.load("y5n_xxs.npy")

In [ ]:
model3_X5_f1_xs.fit(X5, y5n, epochs=200, batch_size=256) 

In [ ]:
model3_X5_f1_xs.save('model3_X5_f1_xs.h5')

In [ ]:
# y5g after load
model3_X5_f1.fit(X5g, y5n, epochs=200, batch_size=256) 

In [ ]:
model3_X5_f1.save('model3_X5_f1.h5')

In [ ]:
model2_X5_f3 = load_model("model2_X5_f3.h5", custom_objects={"get_f1": get_f1 })

In [ ]:
model3_X5_f2 = load_model("model3_X5_f2.h5")

In [ ]:
history = model3_X5_f2.fit(X5g, y5g, epochs=100, batch_size=256) 

In [ ]:
model3_X5_f2.save('model3_X5_f2.h5')

In [ ]:
history.history

In [ ]:
history = model3_X5_f2_xs.fit(X5xs, y5xs, epochs=100, batch_size=256) 

In [ ]:
model3_X5_f2_xs.save('model3_X5_f2_xs.h5')

In [ ]:
np.save("hist2_acc",history.history['accuracy'])
np.save("hist2_loss",history.history['loss'])


In [ ]:
history = model3_X5_f2_xxs.fit(X5, y5n, epochs=100, batch_size=256) 

In [ ]:
model3_X5_f2_xxs.save('model3_X5_f2_xxs.h5')

In [ ]:
np.save("hist3_acc",history.history['accuracy'])
np.save("hist3_loss",history.history['loss'])


In [ ]:
history = model4_X5_f2_xs.fit(X5xs, y5xs, epochs=100, batch_size=256) 

In [ ]:
model4_X5_f2_xs.save('model4_X5_f2_xs.h5')

In [ ]:
np.save("hist5_acc",history.history['accuracy'])
np.save("hist5_loss",history.history['loss'])


In [ ]:
hist2 = np.load("hist2_acc.npy")
hist3 = np.load("hist3_acc.npy")
hist4 = np.load("hist4_acc.npy")
hist5 = np.load("hist5_acc.npy")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(8, 6), dpi=80)
plt.plot(hist2)
plt.plot(hist3)
plt.plot(hist4)
plt.plot(hist5)

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
history = model4_X5_f2_xxs.fit(X5, y5n, epochs=100, batch_size=256) 

In [ ]:
model4_X5_f2_xxs.save('model4_X5_f2_xxs.h5')

In [ ]:
np.save("hist4_acc",history.history['accuracy'])
np.save("hist4_loss",history.history['loss'])


In [ ]:
text = '''
   .   .   .   .   . 
 .   .   .   .   .   
   .   .   x   x   . 
 x   x   x   x   .   
   x   .   x   x   o 
 x   o   o   .   o   
   .   o   o   .   o 
 .   o   o   o   .   
   .   .   .   .   . 
 .   .   .   .   .   B;
'''

In [2]:
text = '''
   x   x   .   .   . 
 o   o   .   .   .   
   .   .   .   .   . 
 .   .   .   .   .   
   .   .   .   .   . 
 .   .   .   .   .   
   .   .   .   .   . 
 o   .   .   .   .   
   .   .   .   .   . 
 .   .   .   .   .   W;
'''

In [ ]:
difPos[22222].reshape(4,10,10)[:2]

In [2]:
text = '''
   x   x   x   x   x 
 x   x   x   x   x   
   x   x   x   .   x 
 x   x   x   x   x   
   o   .   x   .   . 
 .   .   .   .   o   
   .   o   o   .   o 
 o   o   o   o   o   
   o   o   o   o   o 
 o   o   o   o   o   B;
'''

In [78]:
text = '''
   o   o   o   o   o 
 O   O   O   O   O   
   .   .   .   .   . 
 .   .   .   .   .   
   .   .   .   .   . 
 .   .   .   .   .   
   .   .   .   .   . 
 .   .   .   .   .   
   X   X   X   X   X 
 x   x   x   x   x   W;
'''

In [ ]:
convertIDtoPos

In [114]:
s = '.'*5
index = 2
s = s[:index] + 'x' + s[index + 1:]
s

'..x..'

In [107]:
text

'oooooOOOOO..............................XXXXXxxxxxW'

In [106]:
len(text)

51

In [93]:
display_position(pos)

In [94]:
text = 'oooooOOOOO..............................XXXXXxxxxxW'

In [3]:
pos = parse_position(text)
print('eval', eval_position(pos))
print('hash', hash_key(pos), hash(pos))

eval 26
hash 5784582087780755721 5784582087780755721


In [5]:
print(pos)

    x   x   x   x   x  
  x   x   x   x   x  
    x   x   x   .   x  
  x   x   x   x   x  
    o   .   x   .   .  
  .   .   .   .   o  
    .   o   o   .   o  
  o   o   o   o   o  
    o   o   o   o   o  
  o   o   o   o   o  
B


In [7]:
moves = generate_moves(pos)
for move in moves:
    print(print_move(move,pos))

17x26
16x27


In [9]:
print(print_move(moves[1],pos))

16x27


In [11]:
newpos = pos.succ(moves[1])

In [12]:
print(newpos)

    x   x   x   x   x  
  x   x   x   x   x  
    x   x   x   .   x  
  .   x   x   x   x  
    .   .   x   .   .  
  .   x   .   .   o  
    .   o   o   .   o  
  o   o   o   o   o  
    o   o   o   o   o  
  o   o   o   o   o  
W


In [21]:
pos1 = start_position()

In [22]:
pos1

xxxxxxxxxxxxxxxxxxxx..........ooooooooooooooooooooW

In [23]:
pos2 = pos1

In [24]:
pos2

xxxxxxxxxxxxxxxxxxxx..........ooooooooooooooooooooW

In [25]:
pos2.flip()

In [26]:
pos2

xxxxxxxxxxxxxxxxxxxx..........ooooooooooooooooooooB

In [27]:
pos1

xxxxxxxxxxxxxxxxxxxx..........ooooooooooooooooooooB

In [10]:
pos.result()

TypeError: result(): incompatible function arguments. The following argument types are supported:
    1. (self: draughts1.Pos, arg0: draughts1.Side) -> int

Invoked with: O...........................................X.....W

In [99]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

In [100]:
my_x = [np.array([[1.0,2],[3,4]]),np.array([[5.,6],[7,8]])] # a list of numpy arrays
my_y = [np.array([4.]), np.array([2.])] # another list of numpy arrays (targets)

tensor_x = torch.Tensor(my_x) # transform to torch tensor
tensor_y = torch.Tensor(my_y)

my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
my_dataloader = DataLoader(my_dataset) # create your dataloader

In [220]:
idd = '00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000001'

In [103]:
aa = {'data':[1,2,3,4,5],'flag':[True,False,True,False,True]}

In [174]:
idd = '00000000000000000000000000000000000000010000000000100011100000100001000000110000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [221]:
len(idd)

200

In [212]:
ids = []
for i in range(201):
    if i%10!=5:
        ids.append(i)
        if (i%10==0) & (i!=0) & (i!=200):
            ids.append(i)

In [218]:
aa = [1]*200
    bb = np.insert(idn,ids,0).reshape(4,10,10)

In [219]:
bb.shape

(4, 10, 10)

In [216]:
text = '.'*50
idn = np.array([*idd])
# idx = np.where(np.array(idd)=='1')[0]
idn = idn.astype(int)
idn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [142]:
idd.reshape(4,10,5)

array([[['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '1'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0']],

       [['1', '0', '0', '0', '1'],
        ['1', '1', '0', '0', '0'],
        ['0', '0', '1', '0', '0'],
        ['0', '0', '1', '0', '0'],
        ['0', '0', '0', '0', '1'],
        ['1', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '1', '0', '0'],
        ['0', '0', '0', '0', '0']],

       [['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0', '0', '0', '0'],
        ['0', '0

In [140]:
text = '.'*50
idd = np.array([*idd])
idx = np.where(np.array(idd)=='1')[0]
# print(idx)
chrs = ['o','x','O','X']
for i in idx:
    index = i%50
    text = text[:index] + chrs[int(i/50)] + text[index + 1:]
text += 'W'
pos = parse_position(text)

'x...xxx.....x....x......xx.............o..x.......W'

In [132]:
pos = parse_position(text)
print('eval', eval_position(pos))
print('hash', hash_key(pos), hash(pos))

eval -1098
hash 3580854366657979709 3580854366657979709


In [133]:
display_position(pos)

In [104]:
odd = np.array([row['data'] for row in aa])
odd

TypeError: string indices must be integers

In [77]:
# def convertPosTo
board = np.array(np.zeros(200),dtype=int)
ppos = pos
if not ppos.is_white_to_move():
    ppos.flip()
text = print_position(ppos, False, True)
text = np.array([*text[:-1]])

for i,c in enumerate(['o','x','O','X']):
    idx = np.where(np.array(text)==c)[0]
    board[idx+50*i]=1
''.join(map(str,board))

'11111000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000111110000011111000000000000000000000000000000000000000000000000000000000000000000000000000000001111100000'

In [ ]:
pos

In [57]:
idx

(array([0, 1, 2, 3, 4], dtype=int64),)

In [75]:
pos_to_numpy1(ppos).reshape(4,10,10)

array([[[0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
def convertPosToID(pos):

    text = print_position(pos, False, True)

    chr_player = 'o' if pos.is_white_to_move() else 'x'
    chr_other = 'x' if pos.is_white_to_move() else 'o'

    currentplayer_man = []
    other_man = []
    currentplayer_king = []
    other_king = []

    for a,i in enumerate(text[:-1]):
        if a%10<5:
            currentplayer_man.append(0)
            other_man.append(0)
            currentplayer_king.append(0)
            other_king.append(0)
        
        currentplayer_man.append(1 if i==chr_player else 0)
        other_man.append(1 if i==chr_other else 0)
        currentplayer_king.append(1 if i==chr_player.upper() else 0)
        other_king.append(1 if i==chr_other.upper() else 0)
        
        if a%10>=5:
            currentplayer_man.append(0)
            other_man.append(0)
            currentplayer_king.append(0)
            other_king.append(0)
            

    position = np.concatenate((currentplayer_man, other_man, currentplayer_king, other_king), axis=0)
    return np.array(position)

In [31]:
pos.turn()

<Side.White: 0>

In [10]:
wt = pos.turn()

In [6]:
bt = pos.turn()

In [33]:
pos.result(bt)

1

In [ ]:
moves = generate_moves(pos)

In [ ]:
for move in moves:
    print(print_move(move, pos))

In [4]:
display_position(pos)

In [ ]:
def get_move_id(action,pos,move_df):
    move = print_move(action,pos)
    if move_is_capture(action,pos):
           pos_str = move.split('x')
    else:
        pos_str = move.split('-')

    from_ = int(pos_str[0])
    to_ = int(pos_str[1])
    
    move_id = move_df[(move_df['from']==from_)&(move_df['to']==to_)]['move_id'].iloc[0]
    
    return move_id

In [ ]:
for move in moves:
    move_id = get_move_id(move, pos,nmoves)
    print(move_id)
    print(print_move(get_move(move_id,nmoves),pos))

In [ ]:
def get_move(move_id,move_df):
    mv = move_df[move_df['move_id']==move_id]
    from_ = mv['from'].iloc[0]
    to_ = mv['to'].iloc[0]
    
    move_str = str(from_)+'-'+str(to_)
    
    return parse_move(move_str,pos)

In [ ]:
display_position(pos)

In [ ]:
pos.flip()

In [ ]:
for move in moves:
    move_id = get_move_id(move, pos,nmoves)
    print(move_id)
    print(print_move(get_move(move_id,nmoves),pos))

In [ ]:
Scan.set("bb-size", "5")
Scan.update()


EGDB.probe(pos)

In [ ]:
max_depth = 15
max_time = 5
score, move = scan_search(pos, max_depth, max_time)
score

In [ ]:
pos_to_numpy1(pos)

In [ ]:
convertPosToID(pos).reshape(4,10,10)

In [ ]:
pos_to_numpy1(pos).reshape(4,10,10)

In [ ]:
y5_true[3424]

In [ ]:
y5n[3424]

In [ ]:
y5_score[3424]

In [ ]:
X5[3424]

In [ ]:
si = SearchInput()
si.move = True
si.book = False
si.depth = 15
si.nodes = 1000000000000
si.time = 5.0
si.input = True
si.output = OutputType.Terminal

so = SearchOutput()
node = make_node(pos)
search(so, node, si)

In [ ]:
Scan.set("bb-size", "2")
Scan.update()


EGDB.probe(pos)

In [ ]:
display_position(pos)

In [ ]:
pos

In [ ]:
for i in range(1):
    print(i)

In [2]:
import numpy as np
import pandas as pd

In [3]:
board = [[0,1,0,2,0,3,0,4,0,5],
        [6,0,7,0,8,0,9,0,10,0],
        [0,11,0,12,0,13,0,14,0,15],
        [16,0,17,0,18,0,19,0,20,0],
        [0,21,0,22,0,23,0,24,0,25],
        [26,0,27,0,28,0,29,0,30,0],
        [0,31,0,32,0,33,0,34,0,35],
        [36,0,37,0,38,0,39,0,40,0],
        [0,41,0,42,0,43,0,44,0,45],
        [46,0,47,0,48,0,49,0,50,0]]

In [ ]:
np.min([3,2])

In [28]:
from_ = []
to_ = []
move_id = []
count = 0
for i in range(10):
    for j in range(10):
        if board[i][j]>0:
            start = board[i][j]
            # top left
            k = 1
            while (i-k>=0) & (j-k>=0):
                from_.append(board[i][j])
                to_.append(board[i-k][j-k])
                move_id.append(count)
                count+=1
                k+=1
            # top
            k = 1
            while (i-k>=0):
                if board[i-k][j]!=0:
                    from_.append(board[i][j])
                    to_.append(board[i-k][j])
                    move_id.append(count)
                    count+=1
                k+=1
            # top right
            k = 1
            while (i-k>=0) & (j+k<10):
                from_.append(board[i][j])
                to_.append(board[i-k][j+k])
                move_id.append(count)
                count+=1
                k+=1
            # right
            k = 1
            while (j+k<10):
                if board[i][j+k]!=0:
                    from_.append(board[i][j])
                    to_.append(board[i][j+k])
                    move_id.append(count)
                    count+=1
                k+=1
            # left
            k = 1
            while (j-k>=0):
                if board[i][j-k]!=0:
                    from_.append(board[i][j])
                    to_.append(board[i][j-k])
                    move_id.append(count)
                    count+=1
                k+=1
            # bottom left
            k = 1
            while (i+k<10) & (j-k>=0):
                from_.append(board[i][j])
                to_.append(board[i+k][j-k])
                move_id.append(count)
                count+=1
                k+=1
            
            # bottom
            k = 1
            while (i+k<10):
                if board[i+k][j]!=0:
                    from_.append(board[i][j])
                    to_.append(board[i+k][j])
                    move_id.append(count)
                    count+=1
                k+=1
            
            # bottom right
            k = 1
            while (i+k<10) & (j+k<10):
                from_.append(board[i][j])
                to_.append(board[i+k][j+k])
                move_id.append(count)
                count+=1
                k+=1
            

In [13]:
count

2450

In [4]:
count = 0
from_ = []
to_ = []
move_id = []
for i in range(1,51):
    for j in range(1,51):
        from_.append(i)
        to_.append(j)
        move_id.append(count)
        count+=1

In [5]:
capture_moves = pd.DataFrame({'from':from_, 'to':to_, 'move_id':move_id})

In [6]:
capture_moves

,from,to,move_id
0,1,1,0
1,1,2,1
2,1,3,2
3,1,4,3
4,1,5,4
...,...,...,...
2495,50,46,2495
2496,50,47,2496
2497,50,48,2497
2498,50,49,2498


In [9]:
capture_moves.to_csv('capture_moves.csv')

In [8]:
capture_moves[capture_moves['from']==1]

,from,to,move_id
0,1,1,0
1,1,2,1
2,1,3,2
3,1,4,3
4,1,5,4
5,1,6,5
6,1,7,6
7,1,8,7
8,1,9,8
9,1,10,9
